# Categorical Encoding

* In this noebook, we experiment with different encdoers, models, and datasets

In [2]:
# !pip install fasttext dirty_cat category_encoders
# !pip install lightgbm xgboost

In [22]:
from get_data import dataloader
from column_encoder import ColumnEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
import get_data
from joblib import Parallel, delayed
import multiprocessing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction import FeatureHasher
import category_encoders as ce
#from xgboost import XGBClassifier, XGBRegressor

# kc_train = dataloader('data/kaggle_cat_train.csv', "kaggle_cat")
# kc_train.get_input_target()
# kc_test = dataloader('data/kaggle_cat_train.csv', "kaggle_cat")
# kc_test.get_input_target()
# nominal = ['nom_'+str(i) for i in range(0,10)]

## Data

In [23]:
insights = dataloader('data/Insights/insights.csv', "insights")
insights.get_input_target()
X_col = insights.X.columns

In [24]:
def test_train_split(X, y, test_size=0.33, random_state=1):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    sss.get_n_splits(X, y)
    for train_index, test_index in sss.split(X, y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = test_train_split(insights.X, insights.y)

In [25]:
# sample_X_train = X_train[:2000]
# sample_y_train = y_train[:2000]
# sample_X_test = X_test[:200]
# sample_y_test = y_test[:200]

sample_X_train = X_train
sample_y_train = y_train
sample_X_test = X_test
sample_y_test = y_test

### Old experiments

In [44]:
# This will evaluate combinations of encoders with each other. At some point
# in future, not now.
# encoders = [
#     'OneHotEncoder',
#     'BinaryEncoder'
# ]

# param_grid = {f'enc__{col}__encoder_name' : encoders for col in X_col}
# {'enc__modelname__encoder_name': ['OneHotEncoder', 'BinaryEncoder'],
#  'enc__vendor__encoder_name': ['OneHotEncoder', 'BinaryEncoder'],
#  'enc__pmanufacturer__encoder_name': ['OneHotEncoder', 'BinaryEncoder']}

In [8]:
# Create pipelines 
# Dimensionality reduction or breast data if training takes time, start small
# Demo insights data predicting random thing 

enocders_final = [ 
            'OneHotEncoder',
            'SimilarityEncoder',
            'NgramNaiveFisherKernel', #U
            'BinaryEncoder',
            'HashingEncoder',
            'NgramsCountVectorizer', #U
            'NgramsTfIdfVectorizer', #U
            'WordNMF', #U
            'WordNgramsTfIdfVectorizer', #U
                 
]

# Contrast encoder
# Package can apply to unsupervised but in example it is only for ordinal data
# In blog, it says supervised association

In [2]:
encoders = ['OneHotEncoder',
            'SimilarityEncoder',
            'NgramNaiveFisherKernel',
            'BackwardDifferenceEncoder',
            'BinaryEncoder',
            #'HashingEncoder',
            'HelmertEncoder',
            'SumEncoder',
            'PolynomialEncoder',
            'BaseNEncoder',
            'NgramsCountVectorizer',
            'NgramsTfIdfVectorizer',
            'WordNMF',
            'WordNgramsTfIdfVectorizer',
]

not_working =  [ 
            #'ngrams_hot_vectorizer',
            #'NgramsLDA',
            # 'NgramsMultinomialMixture',
            
            #takes too long
            # 'NMF' 
            # 'AdHocNgramsMultinomialMixture',
            #'AdHocIndependentPDF',
    
            # These have some kmeans initialization errors cuz of matrix dims
            #'OnlineGammaPoissonFactorization',
            # 'OnlineGammaPoissonFactorization2',
            # 'OnlineGammaPoissonFactorization3',
            # 'OnlineGammaPoissonFactorization4',
            # 'WordOnlineGammaPoissonFactorization',
            # 'OnlineGammaPoissonFactorization_fast',
            
            # Other errors
            # 'MDVEncoder',
            # 'MinHashEncoder',
    
            # 'PretrainedFastText',
            # 'PretrainedFastText_hu',
    ]

supervised = [
    #'TargetEncoder', 'LeaveOneOutEncoder',
    ]



def exp_encode(enc):
    kc_train.col_encoders = {col:  ColumnEncoder(enc) 
        for col in kc_train.X.columns}
    
    nominal = ['nom_'+str(i) for i in range(0,10)]
    kc_train_sample_X = kc_train.X.iloc[:3000][nominal]
    kc_test_sample_X = kc_test.X.iloc[:500][nominal]
    
    cT = ColumnTransformer([(col, kc_train.col_encoders[col], col) for col in kc_train_sample_X])
    cT.fit(kc_train_sample_X)
    return cT


num_cores = multiprocessing.cpu_count()
results = Parallel(backend="multiprocessing", n_jobs=num_cores)(delayed(exp_encode)(enc) for enc in encoders)
print(results)


# for enc in encoders:
#     print(enc)
#     kc_train.col_encoders = {col:  ColumnEncoder(enc) 
#         for col in kc_train.X.columns}
    
#     nominal = ['nom_'+str(i) for i in range(0,10)]
    
#     kc_train_sample_X = kc_train.X.iloc[:300][nominal]
#     kc_train_sample_y = kc_train.y.iloc[:300]
#     kc_test_sample_X = kc_test.X.iloc[:50][nominal]
#     kc_test_sample_y = kc_test.y.iloc[:50]
    
#     cT = ColumnTransformer([(col, kc_train.col_encoders[col], col) for col in kc_train_sample_X])
#     cT.fit(kc_train_sample_X)
#     #x_out = cT.fit_transform(kc_train_sample_X)
    
#     # x_test_out = cT.transform(kc_test_sample_X)
    
#     # lgbm = LGBMClassifier()
#     # lgbm.fit(x_out, kc_train_sample_y)
    
#     # print(classification_report(y_pred = lgbm.predict(x_test_out), y_true = kc_test_sample_y))
            

/opt/app-root/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/app-root/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.nmf module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/app-root/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Pleas

[ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('nom_0',
                                 ColumnEncoder(categories='auto', clf_type=None,
                                               dtype=<class 'numpy.float64'>,
                                               encoder_name='OneHotEncoder',
                                               handle_unknown='ignore',
                                               n_components=None,
                                               ngram_range=(2, 4),
                                               reduction_method=None),
                                 'nom_0'),
                                ('nom_1',
                                 ColumnEncoder(categories='aut...
                                               dtype=<class 'numpy.float64'>,
                                               encoder_name='OneHotEncoder',
                        

In [24]:
shapes = [enc.transform(kc_test_sample_X) for enc in results[5:]]

In [25]:
print([i.shape for i in shapes])

[(500, 5685), (500, 5685), (500, 5685), (500, 73), (500, 47562), (500, 47562), (500, 5686), (500, 5686)]


In [35]:
np.hstack((results[5:8], results[10:])).shape

(6,)

In [ ]:
def exp_train(enc):
    kc_train_sample_X = kc_train.X.iloc[:3000][nominal]
    kc_train_sample_y = kc_train.y.iloc[:3000]
    kc_test_sample_X = kc_test.X.iloc[:500][nominal]
    X_out, X_test_out = enc.transform(kc_train_sample_X), enc.transform(kc_test_sample_X)
    
    lgbm = LGBMClassifier()
    lgbm.fit(X_out, kc_train_sample_y)
    return lgbm
 
    
    
# num_cores = multiprocessing.cpu_count()
# models = Parallel(backend="multiprocessing", n_jobs=num_cores)(delayed(exp_train)(enc) for enc in results[:5])
# print(models)
# np.save("data/models.npy", models)

remaining = np.hstack((results[5:8], results[10:])) #removing high dimensional encoders to reduce computation
num_cores = multiprocessing.cpu_count()
models_2 = Parallel(backend="multiprocessing", n_jobs=num_cores)(delayed(exp_train)(enc) for enc in remaining)
print(models_2)
np.save("data/models_2.npy", models)

In [6]:
import numpy as np
#np.save("data/results.npy", results)
results = np.load("data/results.npy", allow_pickle=True)
models = np.load("data/models.npy", allow_pickle=True)
#models_2 = np.load("data/models.npy", allow_pickle=True)

In [7]:
kc_test_sample_X = kc_test.X.iloc[:500][nominal]
kc_test_sample_y = kc_test.y.iloc[:500]

# def classification_reports(encoder, model):
#     X_test_out = encoder.transform(kc_test_sample_X)
#     return classification_report(y_pred = model.predict(X_test_out), y_true = kc_test_sample_y)
    

# crs = Parallel(backend="multiprocessing", n_jobs=num_cores)\
# (delayed(classification_reports)(results[e], model) for e, model in enumerate(models))
# # for lgbm in models:
0#     print(lgbm, classification_report(y_pred = lgbm.predict(x_test_out), y_true = kc_test_sample_y))   

In [ ]:
X_test_out = results[0].transform(kc_test_sample_X)
print(classification_report(y_pred = models[0].predict(X_test_out), y_true = kc_test_sample_y))

In [ ]:
X_test_out = results[1].transform(kc_test_sample_X)
print(classification_report(y_pred = models[1].predict(X_test_out), y_true = kc_test_sample_y))

In [ ]:
X_test_out_2 = results[2].transform(kc_test_sample_X)
#classification_report(y_pred = models[2].predict(X_test_out), y_true = kc_test_sample_y)

In [ ]:
print(classification_report(y_pred = models[2].predict(X_test_out_2), y_true = kc_test_sample_y))

In [ ]:
X_test_out = results[3].transform(kc_test_sample_X)
print(classification_report(y_pred = models[3].predict(X_test_out), y_true = kc_test_sample_y))

In [ ]:
X_test_out = results[4].transform(kc_test_sample_X)
print(classification_report(y_pred = models[4].predict(X_test_out), y_true = kc_test_sample_y))

In [ ]:
SimilarityEncoder